In [1]:
import pyarrow as pa
from pyarrow import json
import liten as ten

<frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.6 of module 'liten.lib' does not match runtime version 3.8
<frozen importlib._bootstrap>:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject


In [2]:
tc = ten.Cache()

In [3]:
reviews_file='/mnt/c/Users/hkver/Documents/dbai/datasets/Musical_Instruments_5.json'
reviews_pa = pa.json.read_json(input_file=reviews_file)
reviews_table = tc.add_table("reviews_all", reviews_pa, tc.FactTable)
reviews_table

'reviews_all'

In [4]:
reviews_pa = reviews_pa.drop(['reviewerID','asin','helpful','unixReviewTime','reviewTime'])
reviews_table = tc.add_table("reviews", reviews_pa, tc.FactTable)
tc.slice('reviews',0,5).to_pandas()

,reviewerName,reviewText,overall,summary
0,"cassandra tu ""Yeah, well, that's just like, u...","Not much to write about here, but it does exac...",5.0,good
1,Jake,The product does exactly as it should and is q...,5.0,Jake
2,"Rick Bennette ""Rick Bennette""",The primary job of this device is to block the...,5.0,It Does The Job Well
3,"RustyBill ""Sunday Rocker""",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY
4,SEAN MASLANKA,This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.


In [5]:
from textblob import TextBlob

In [6]:
reviews_pa = tc.slice('reviews',0,12)
review_texts = reviews_pa.to_pandas()['reviewText']       

In [7]:
review_texts

0     Not much to write about here, but it does exac...
1     The product does exactly as it should and is q...
2     The primary job of this device is to block the...
3     Nice windscreen protects my MXL mic and preven...
4     This pop filter is great. It looks and perform...
5     So good that I bought another one.  Love the h...
6     I have used monster cables for years, and with...
7     I now use this cable to run from the output of...
8     Perfect for my Epiphone Sheraton II.  Monster ...
9     Monster makes the best cables and a lifetime w...
10    Monster makes a wide array of cables, includin...
11    I got it to have it if I needed it. I have fou...
Name: reviewText, dtype: object

In [8]:
review_sentiments = []
review_sentiments_text = []
for review_text in review_texts:
    blob = TextBlob(review_text)
    review_sentiments.append(blob.sentiment.polarity)
    if (blob.sentiment.polarity > 0.5):
        review_sentiment_text = "+++"
    elif (blob.sentiment.polarity > 0.25):
        review_sentiment_text = "++"
    elif (blob.sentiment.polarity >= 0):
        review_sentiment_text = "+"
    elif (blob.sentiment.polarity > -0.25):
        review_sentiment_text = "-"
    elif (blob.sentiment.polarity > -0.5):
        review_sentiment_text = "--"
    else:
        review_sentiment_text = "---"
    review_sentiments_text.append(review_sentiment_text)

In [9]:
reviews_pa = reviews_pa.append_column('sentiment',pa.array(review_sentiments))
reviews_pa = reviews_pa.append_column('sentiment_text',pa.array(review_sentiments_text))

In [10]:
reviews_pa.to_pandas()

,reviewerName,reviewText,overall,summary,sentiment,sentiment_text
0,"cassandra tu ""Yeah, well, that's just like, u...","Not much to write about here, but it does exac...",5.0,good,0.250000,+
1,Jake,The product does exactly as it should and is q...,5.0,Jake,0.052778,+
2,"Rick Bennette ""Rick Bennette""",The primary job of this device is to block the...,5.0,It Does The Job Well,0.167500,+
3,"RustyBill ""Sunday Rocker""",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,0.200000,+
4,SEAN MASLANKA,This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,0.800000,+++
5,"Bill Lewey ""blewey""",So good that I bought another one. Love the h...,5.0,The Best Cable,0.338889,++
6,Brian,"I have used monster cables for years, and with...",5.0,Monster Standard 100 - 21' Instrument Cable,0.333333,++
7,"Fender Guy ""Rick""",I now use this cable to run from the output of...,3.0,Didn't fit my 1996 Fender Strat...,0.006540,+
8,"G. Thomas ""Tom""",Perfect for my Epiphone Sheraton II. Monster ...,5.0,Great cable,0.500000,++
9,Kurt Robair,Monster makes the best cables and a lifetime w...,5.0,Best Instrument Cables On The Market,0.330000,++
